In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.9 MB/s eta 0:00:00


In [3]:
!pip install PyPDF2
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.9 MB/s eta 0:00:00


In [10]:
!curl https://loca.lt/mytunnelpassword

35.245.53.111

In [11]:
!streamlit run app.py &>/content/logs.txt &

In [12]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴your url is: https://many-bees-train.loca.lt
^C


In [9]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import re

# Function to extract text from PDF
def extract_text_from_pdf(file):
    pdf = PdfReader(file)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

# Function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Function to rank resumes based on job description
def rank_resumes(job_description, resumes):
    # Preprocess job description and resumes
    job_description_processed = preprocess_text(job_description)
    resumes_processed = [preprocess_text(resume) for resume in resumes]

    # Combine job description with resumes
    documents = [job_description_processed] + resumes_processed
    vectorizer = TfidfVectorizer(stop_words='english').fit_transform(documents)
    vectors = vectorizer.toarray()

    # Calculate cosine similarity
    job_description_vector = vectors[0]
    resume_vectors = vectors[1:]
    cosine_similarities = cosine_similarity([job_description_vector], resume_vectors).flatten()

    # Normalize scores to 0-100 scale
    scores = (cosine_similarities * 100).round(2)

    return scores

# Streamlit app
st.set_page_config(page_title="AI Resume Screening & Ranking", layout="wide")
st.title("📄 AI Resume Screening & Candidate Ranking System")

# Job description input
st.sidebar.header("🔍 Job Description")
job_description = st.sidebar.text_area("Enter the job description", height=150)

# File uploader
st.sidebar.header("📂 Upload Resumes")
uploaded_files = st.sidebar.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)

if uploaded_files and job_description:
    st.header("📊 Ranking Resumes")

    # Extract text from uploaded resumes
    resumes = []
    for file in uploaded_files:
        text = extract_text_from_pdf(file)
        resumes.append(text)

    # Rank resumes
    scores = rank_resumes(job_description, resumes)

    # Create results DataFrame
    results = pd.DataFrame({
        "Resume": [file.name for file in uploaded_files],
        "Score": scores
    })
    results = results.sort_values(by="Score", ascending=False)

    # Display table
    st.subheader("Resume Scores")
    st.dataframe(results, height=400)

    # Display bar chart
    st.subheader("Resume Match Scores")
    fig = px.bar(results, x="Resume", y="Score", text="Score", labels={"Score": "Match Score"}, color="Score",
                 color_continuous_scale=px.colors.sequential.Viridis)
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(xaxis_title="Resume", yaxis_title="Match Score", yaxis_range=[0, 100])
    st.plotly_chart(fig, use_container_width=True)

    # Highlight the best resume
    best_resume = results.iloc[0]
    st.success(f"🏆 Best Resume: **{best_resume['Resume']}** with a score of **{best_resume['Score']:.2f}**")

Overwriting app.py
